#Pydantic Example: Creating a Simple Custom Class



In [ ]:
from datetime import datetime

from pydantic import BaseModel, PositiveInt

class User(BaseModel):

    id: int

    name: str = 'John Doe'

    signup_ts: datetime | None

    tastes: dict[str, PositiveInt]

external_data = {

    'id': 123,

    'signup_ts': '2019-06-01 12:22',

    'tastes': {

        'wine': 9,

        b'cheese': 7,

        'cabbage': '1',

    },

}

user = User(**external_data)

print(user.id)

#> 123

print(user.model_dump())

"""

{

    'id': 123,

    'name': 'John Doe',

    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),

    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},

}

"""

In [ ]:
# continuing the above example...

from datetime import datetime

from pydantic import BaseModel, PositiveInt, ValidationError

class User(BaseModel):

    id: int

    name: str = 'John Doe'

    signup_ts: datetime | None

    tastes: dict[str, PositiveInt]

external_data = {'id': 'not an int', 'tastes': {}}

try:

    User(**external_data)

except ValidationError as e:

    print(e.errors())

    """

    [

        {

            'type': 'int_parsing',

            'loc': ('id',),

            'msg': 'Input should be a valid integer, unable to parse string as an integer',

            'input': 'not an int',

            'url': 'https://errors.pydantic.dev/2/v/int_parsing',

        },

        {

            'type': 'missing',

            'loc': ('signup_ts',),

            'msg': 'Field required',

            'input': {'id': 'not an int', 'tastes': {}},

            'url': 'https://errors.pydantic.dev/2/v/missing',

        },

    ]

    """

#How to Build an Agent Using PydanticAI?

In [ ]:
pip install pydantic-ai pydantic

In [ ]:
from dataclasses import dataclass

from bank_database import DatabaseConn  # Custom database connection

@dataclass

class SupportDependencies:

    customer_id: int

    db: DatabaseConn

In [ ]:
from pydantic import BaseModel, Field

class SupportResult(BaseModel):

    support_advice: str = Field(description="Advice returned to the customer")

    block_card: bool = Field(description="Whether to block the customer's card")

    risk: int = Field(description="Risk level of query", ge=0, le=10)

In [ ]:
from pydantic_ai import Agent

support_agent = Agent(

    'openai:gpt-4o',  # Specify the AI model

    deps_type=SupportDependencies,

    result_type=SupportResult,

    system_prompt=(

        "You are a support agent in our bank, give the customer support "

        "and judge the risk level of their query."

    ),

)

In [ ]:
from pydantic_ai import RunContext

@support_agent.system_prompt

async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:

    customer_name = await ctx.deps.db.customer_name(id=ctx.deps.customer_id)

    return f"The customer's name is {customer_name!r}"

In [ ]:
@support_agent.tool

async def customer_balance(ctx: RunContext[SupportDependencies], include_pending: bool) -> float:

    """Returns the customer's current account balance."""

    return await ctx.deps.db.customer_balance(

        id=ctx.deps.customer_id,

        include_pending=include_pending,

    )

In [ ]:
async def main():

    deps = SupportDependencies(customer_id=123, db=DatabaseConn())

    # Query about account balance

    result = await support_agent.run("What is my balance?", deps=deps)

    print(result.data)

    # Output:

    # support_advice='Hello John, your current account balance, including pending transactions, is $123.45.'

    # block_card=False risk=1

    # Query about lost card

    result = await support_agent.run("I just lost my card!", deps=deps)

    print(result.data)

    # Output:

    # support_advice="I'm sorry to hear that, John. We are temporarily blocking your card to prevent unauthorized transactions."

    # block_card=True risk=8